##5.1 Layers and Blocks


In [ ]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.02406494, -0.21575084,  0.284638  , -0.26737177, -0.149414  ,
        -0.00654668, -0.4775137 , -0.03806105,  0.04162936,  0.30456537],
       [ 0.1924675 , -0.46334952,  0.21149538, -0.31973386, -0.22635445,
        -0.21373498, -0.3224384 ,  0.03797204,  0.0276132 , -0.06042683]],
      dtype=float32)>

In [ ]:
!pip install tensorflow

     |████████████████████████████████| 462 kB 26.2 MB/s 


### 5.1.1. A Custom Block

In [ ]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Model` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def call(self, X):
        return self.out(self.hidden((X)))

In [ ]:
net = MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.15583804, -0.4286663 , -0.03436851, -0.15545624,  0.10530031,
        -0.26984227, -0.08079828, -0.39965156,  0.21935347, -0.24504113],
       [-0.11888622, -0.02584201,  0.04966003,  0.05296024,  0.14277372,
        -0.2710023 , -0.038265  , -0.5867928 ,  0.396201  , -0.24767761]],
      dtype=float32)>

### 5.1.2. The Sequential Block

In [ ]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [ ]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.07596566, -0.3520943 ,  0.2806658 , -0.13451932, -0.20868804,
        -0.3480026 ,  0.37253967,  0.47441107,  0.12159334,  0.06263323],
       [ 0.04211479, -0.19824073,  0.15802379, -0.23124363, -0.27027398,
        -0.21336943,  0.24694929,  0.49771458,  0.14753363, -0.10391346]],
      dtype=float32)>

### 5.1.3. Executing Code in the Forward Propagation Function

In [ ]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with `tf.constant` are not updated
        # during training (i.e., constant parameters)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the `relu` and
        # `matmul` functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [ ]:
net = FixedHiddenMLP()
net(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.95621747>

In [ ]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())
chimera(X)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5673848>

## 5.2 Parameter Management

In [ ]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.60277617],
       [0.4408723 ]], dtype=float32)>

### 5.2.1. Parameter Access

In [ ]:
print(net.layers[2].weights)

[<tf.Variable 'dense_13/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.5162623 ],
       [ 0.92296314],
       [ 0.28223372],
       [-0.6950456 ]], dtype=float32)>, <tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


#### 5.2.1.1. Targeted Parameters

In [ ]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_13/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


#### 5.2.1.2. All Parameters at Once

In [ ]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_12/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.32645446, -0.49626824, -0.6665583 , -0.8555467 ],
       [-0.56307125,  0.5456403 , -0.02340388, -0.03082305],
       [-0.57156324,  0.4364639 , -0.56837606, -0.49148563],
       [-0.77329886,  0.35663348,  0.2065162 , -0.5387859 ]],
      dtype=float32)>, <tf.Variable 'dense_12/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.32645446, -0.49626824, -0.6665583 , -0.8555467 ],
       [-0.56307125,  0.5456403 , -0.02340388, -0.03082305],
       [-0.57156324,  0.4364639 , -0.56837606, -0.49148563],
       [-0.77329886,  0.35663348,  0.2065162 , -0.5387859 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 0.5162623 ],
       [ 0.92296314],
       [ 0.28223372],
       [-0.6950456 ]], dtype=float32), array([0.], dtype=float32)]


#### 5.2.1.3. Collecting Parameters from Nested Blocks

In [ ]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.0486657 ],
       [-0.08791585]], dtype=float32)>

In [ ]:
print(rgnet.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (2, 4)                    80        
                                                                 
 dense_18 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_15/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

### 5.2.2. Parameter Initialization

#### 5.2.2.1. Built-in Initialization

In [ ]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_19/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.00122025,  0.00131971, -0.00393437, -0.00427214],
        [ 0.00614608,  0.00869299,  0.00058501,  0.01082132],
        [-0.00195795, -0.01094769, -0.00334236, -0.00970782],
        [ 0.01336065, -0.00076073,  0.00528313, -0.01311768]],
       dtype=float32)>,
 <tf.Variable 'dense_19/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [ ]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_21/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_21/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [ ]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_23/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.03259271, -0.6122806 , -0.6565229 ,  0.4345768 ],
       [ 0.37506753, -0.10443437,  0.3769322 , -0.28130674],
       [ 0.1585465 , -0.5133585 ,  0.6919758 ,  0.12881702],
       [-0.13847947, -0.3328951 ,  0.19812804, -0.48763812]],
      dtype=float32)>
<tf.Variable 'dense_24/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[42.],
       [42.],
       [42.],
       [42.]], dtype=float32)>


#### 5.2.2.2. Custom Initialization

In [ ]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_27/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 6.907009 ,  0.       , -0.       ,  9.16346  ],
       [-6.8709254,  9.123123 , -0.       , -6.185539 ],
       [-0.       ,  0.       , -8.637695 ,  0.       ],
       [ 0.       , -9.747543 ,  0.       ,  0.       ]], dtype=float32)>


In [ ]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_27/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.       ,  1.       ,  1.       , 10.16346  ],
       [-5.8709254, 10.123123 ,  1.       , -5.185539 ],
       [ 1.       ,  1.       , -7.6376953,  1.       ],
       [ 1.       , -8.747543 ,  1.       ,  1.       ]], dtype=float32)>

### 5.2.3. Tied Parameters¶

In [ ]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


## 5.3. Deferred Initialization

### 5.3.1. Instantiating a Network

In [ ]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [ ]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [ ]:
X = tf.random.uniform((2, 20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

## 5.4. Custom Layers

### 5.4.1. Layers without Parameters

In [ ]:
import tensorflow as tf


class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [ ]:
layer = CenteredLayer()
layer(tf.constant([1, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [ ]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [ ]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=4.8894435e-09>

### 5.4.2. Layers with Parameters

In [ ]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [ ]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.05980575, -0.01197818,  0.04616862],
        [-0.12864971,  0.07238882, -0.0232415 ],
        [ 0.00779543,  0.02651612, -0.00922562],
        [-0.02197372, -0.09296186,  0.09228   ],
        [ 0.02961118,  0.02127842,  0.05830185]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [ ]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.00647273, 0.0325605 , 0.04417397],
       [0.01242206, 0.02690992, 0.05752953]], dtype=float32)>

In [ ]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.02685481],
       [0.02324918]], dtype=float32)>

## 5.5. File I/O

### 5.5.1. Loading and Saving Tensors

In [ ]:
import numpy as np
import tensorflow as tf

x = tf.range(4)
np.save('x-file.npy', x)

In [ ]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [ ]:
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [ ]:
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

### 5.5.2. Loading and Saving Model Parameters

In [ ]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net = MLP()
X = tf.random.uniform((2, 20))
Y = net(X)

In [ ]:
net.save_weights('mlp.params')

In [ ]:
clone = MLP()
clone.load_weights('mlp.params')

In [ ]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>